
200 examples in the public dataset leaves very little room for training!

Using `gpt-3.5-turbo` I created another 500 high quality examples at my expense [that I share freely here](https://www.kaggle.com/datasets/radek1/additional-train-data-for-llm-science-exam). They are what (as of now) pushes this notebook to the highest achieved score across the public notebooks (`0.723`)!

If you find the additional training examples useful, please upvote the dataset! 😊

👉 [additional train data for LLM Science Exam 🥳](https://www.kaggle.com/datasets/radek1/additional-train-data-for-llm-science-exam)

Thank you! Appreciate your help! 🙏🙏🙏

This notebook builds on a [notebook](https://www.kaggle.com/code/leonidkulyk/lb-0-709-llm-se-deberta-v3-large-i-1k-wiki) by LEONID KULYK. Among some of the changes are:

* use of a new high quality dataset for training
* modified training procedure carried out in the notebook
* general streamlining of code/training for readability

**A couple of related resources you might find useful:**

* [📊 15k high-quality train examples 🏆🔥🚀](https://www.kaggle.com/datasets/radek1/15k-high-quality-examples) - another 15 000 examples I created to help you grow that train/validation set of yours and improve results
* [📊 Best Open Source LLM Starter Pack 🧙🚀](https://www.kaggle.com/datasets/radek1/best-llm-starter-pack) -- the largest (and best) open source model I managed to run on Kaggle!
* [Science Exam Trained Model Weights 🚀](https://www.kaggle.com/datasets/radek1/science-exam-trained-model-weights)

# Preprocessing the dataset

In [ ]:
from typing import Optional, Union
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from dataclasses import dataclass
from transformers import AutoTokenizer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoModel

deberta_v3_large = '/kaggle/input/deberta-v3-large-hf-weights'

We begin by loading and processing the train data.

In [ ]:
df_train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
df_train = df_train.drop(columns="id")
df_train.shape

Let's add another 500 examples to the train set!

In [ ]:
df_train = pd.concat([
    df_train,
    pd.read_csv('/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv'),
])
df_train.reset_index(inplace=True, drop=True)
df_train.shape

Now that we have gone from 200 -> 700 train examples, let us preprocess the data and begin training.

In [ ]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k,v in option_to_index.items()}

def preprocess(example):
    first_sentence = [example['prompt']] * 5
    second_sentences = [example[option] for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation=False)
    tokenized_example['label'] = option_to_index[example['answer']]
    
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

We first create a HuggingFace `Dataset`.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(deberta_v3_large)

dataset = Dataset.from_pandas(df_train)
dataset

And let us now preprocess the examples for training.

In [ ]:
tokenized_dataset = dataset.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
tokenized_dataset

# Training

In [ ]:
training_args = TrainingArguments(
    warmup_ratio=0.8,
    learning_rate=5e-6,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    report_to='none',
    output_dir='.',
)

model = AutoModelForMultipleChoice.from_pretrained(deberta_v3_large)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_dataset,
)

trainer.train()

# Predicting on the test set

Now that we have trained our model, let us predict on the test set.

In [ ]:
test_df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')
test_df['answer'] = 'A' # dummy answer that allows us to preprocess the test dataset just like we preprocessed the train dataset

tokenized_test_dataset = Dataset.from_pandas(test_df.drop(columns=['id'])).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E'])

In [ ]:
test_predictions = trainer.predict(tokenized_test_dataset).predictions
test_predictions[:4]

The predictions are values output from the last layer of our neural network.

Let's obtain the predicted answer ids by sorting them from largest to the smallest.

In [ ]:
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_ids[:3]

Let us now assign a letter corresponding to each predicted id (0 -> 'A', 1 -> 'B', etc). 

In [ ]:
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
predictions_as_answer_letters[:3]

And let us now go from this representation to outputting a string with 3 highest rated answers seperated by a space.

In [ ]:
predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]
predictions_as_string[:3]

And we are done! 🥳

Let us now output our submission.

In [ ]:
submission = test_df[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)

pd.read_csv('submission.csv').head()

I hope you enjoyed this notebook!

If you found it useful, please upvote 👉 [the corresponding dataset with 500 new training examples](https://www.kaggle.com/datasets/radek1/additional-train-data-for-llm-science-exam) 👈

Thank you, appreciate your help! 🙏😊

Thank you for reading and happy Kaggling!